# Run all 

In [4]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
scrapers = [SenterpriseScraper()] #Cindoe

#scrapers = []

nr_ads_pre = len(load_local_data())

for s in scrapers:
    response = s.request_status()
    scraped_payload_dict = s.scrape_jobs_payloads_dict(response)
    
    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_payload_dict = dict(zip(old_payload_data['id'], old_payload_data['raw_payload']))
    old_bronze_data = load_local_data()
    
    new_payload_dict = s.return_new_payloads(new_dict=scraped_payload_dict, old_dict=old_payload_dict)
    new_bronze_data = s.parse_bronze_data(new_payload_dict)

    updated_payload_dict = s.concat_dicts(new_payload_dict, old_payload_dict)
    updated_payload_data = pd.DataFrame(list(updated_payload_dict.items()), columns=['id','raw_payload'])
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)

    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
updated_bronze_data.loc[updated_bronze_data['site'] ==s.__class__.site]



Senterprise > Response: 200
Senterprise > Nmr of scraped adds: 8
Senterprise > Parsing bronze data: 8
Total number of new added jobs: 0


,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new
806,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6504216-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6504216-konsu...,2025-09-25 14:29:17,True
807,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6487493-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6487493-konsu...,2025-09-25 14:29:17,True
808,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6484496-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6484496-konsu...,2025-09-25 14:29:17,True
809,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6475370-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6475370-konsu...,2025-09-25 14:29:17,True
810,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6464556-konsu...,None,None,None,Stockholm,None,https://jobb.senterprise.se/jobs/6464556-konsu...,2025-09-25 14:29:17,True
811,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6438444-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6438444-konsu...,2025-09-25 14:29:17,True
812,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6438010-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6438010-konsu...,2025-09-25 14:29:17,True
813,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6421292-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6421292-konsu...,2025-09-25 14:29:17,True


# Version 2

In [19]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
#scrapers = [TingentScraper()] #Cindoe
#scrapers = []

nr_ads_pre = len(load_local_data())

for s in scrapers:
    response = s.request_status()
    job_payloads = s.extract_job_payloads(response)
    scraped_data = s.scrape_all_jobs(job_payloads)

    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_bronze_data = load_local_data()
    old_data = old_bronze_data.merge(old_payload_data, on="id")
    print('old data:', len(old_data))
    
    new_scraped_data = s.return_new_rows(scraped_data, old_bronze_data)
    print('new data:', len(new_scraped_data))

    updated_data = AbstractScraper.set_dtypes(pd.concat([old_data, new_scraped_data], ignore_index=True))
    print('updated data:', len(updated_data))
    
    updated_bronze_data = updated_data[AbstractScraper.bronze_columns]
    updated_payload_data = updated_data[AbstractScraper.payload_columns]
    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
updated_data.loc[updated_data['site'] ==s.site]

Afry > Response: 200
Afry > Nmr of scraped adds: 70
old data: 806
Afry > Nmr of new adds: 7
new data: 7
updated data: 813
Aliant > Response: 200
Aliant > Nmr of scraped adds: 10
old data: 806
Aliant > Nmr of new adds: 2
new data: 2
updated data: 808
A Society > Response: 200
A Society > Nmr of scraped adds: 121
old data: 806
A Society > Nmr of new adds: 35
new data: 35
updated data: 841
Cinode > Response: 200
Cinode > Nmr of scraped adds: 20
old data: 806
Cinode > Nmr of new adds: 20
new data: 20
updated data: 826
Combitech > Response: 200
Combitech > Nmr of scraped adds: 41
old data: 806
Combitech > Nmr of new adds: 17
new data: 17
updated data: 823
Emagine > Response: 200
Emagine > Nmr of scraped adds: 52
old data: 806
Emagine > Nmr of new adds: 15
new data: 15
updated data: 821
Ework > Response: 200
Ework > Nmr of scraped adds: 108
old data: 806
Ework > Nmr of new adds: 85
new data: 85
updated data: 891
ITC Network > Response: 200
ITC Network > Nmr of scraped adds: 33
old data: 806


,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new,raw_payload
369,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,Upgraded-https://upgraded.se/konsultuppdrag/12...,Lime utvecklare,Lime utvecklare,2025-09-29,"Landskrona, SE",Distansarbete: Hybrid,https://upgraded.se/konsultuppdrag/127399-lime...,2025-09-15 13:53:36,False,"<tr class=""konsultuppdrag__table-row"" onclick=..."
370,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,Upgraded-https://upgraded.se/konsultuppdrag/12...,Expert inom Digitaliseringssamordning till Fra...,nan,2025-09-18,Hässleholm,nan,https://upgraded.se/konsultuppdrag/127359-expe...,2025-09-15 13:53:36,False,"<tr class=""konsultuppdrag__table-row"" onclick=..."
371,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,Upgraded-https://upgraded.se/konsultuppdrag/12...,DevOps Engineer,DevOps Engineer,2025-09-18,Södertälje,"Distansarbete: Hybrid, with 2–4 mandatory offi...",https://upgraded.se/konsultuppdrag/127325-devo...,2025-09-15 13:53:36,False,"<tr class=""konsultuppdrag__table-row"" onclick=..."
372,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,Upgraded-https://upgraded.se/konsultuppdrag/12...,Business Change Management Strategy and framew...,Change Manager,2025-09-19,Södertälje,nan,https://upgraded.se/konsultuppdrag/127297-busi...,2025-09-15 13:53:36,False,"<tr class=""konsultuppdrag__table-row"" onclick=..."
373,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,Upgraded-https://upgraded.se/konsultuppdrag/12...,Project Manager,Project Manager,2025-09-18,Södertälje,nan,https://upgraded.se/konsultuppdrag/127266-proj...,2025-09-15 13:53:36,False,"<tr class=""konsultuppdrag__table-row"" onclick=..."
...,...,...,...,...,...,...,...,...,...,...,...,...
864,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129328-test...,Testare,Testare,2025-09-29,Stockholm,,https://upgraded.se/konsultuppdrag/129328-test...,2025-09-26 14:37:07,True,"<td class=""konsultuppdrag-column-1"" style="""">\..."
865,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129327-ma-s...,MA-specialist (Marketing Specialist),Marketing specialist,2025-09-30,Stockholm,,https://upgraded.se/konsultuppdrag/129327-ma-s...,2025-09-26 14:37:07,True,"<td class=""konsultuppdrag-column-1"" style="""">\..."
866,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129211-simu...,Simulator Developer,Software Engineer,2025-09-29,"Göteborg, SE",,https://upgraded.se/konsultuppdrag/129211-simu...,2025-09-26 14:37:07,True,"<td class=""konsultuppdrag-column-1"" style="""">\..."
867,Upgraded-https://upgraded.se/konsultuppdrag/12...,Upgraded,https://upgraded.se/konsultuppdrag/129167-krav...,Kravanalytiker,Kravanalytiker,2025-10-01,Norrköping,Distansarbete: Hybrid,https://upgraded.se/konsultuppdrag/129167-krav...,2025-09-26 14:37:07,True,"<td class=""konsultuppdrag-column-1"" style="""">\..."


## Load bronze table 

In [ ]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
df = bronze_data.copy()
print(len(df['work_location'].unique()))
df['work_location'].unique()
display(df)
